## Importing Libraries

In [41]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

## Loading Data

In [65]:
order=pd.read_csv('2019 Winter Data Science Intern Challenge Data Set - Sheet1.csv')
order

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at
0,1,53,746,224,2,cash,2017-03-13 12:36:56
1,2,92,925,90,1,cash,2017-03-03 17:38:52
2,3,44,861,144,1,cash,2017-03-14 4:23:56
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11
...,...,...,...,...,...,...,...
4995,4996,73,993,330,2,debit,2017-03-30 13:47:17
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16
4997,4998,56,867,351,3,cash,2017-03-19 5:42:42
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18


## Checking the Existence of Missing Values

In [66]:
order.isnull().sum()

order_id          0
shop_id           0
user_id           0
order_amount      0
total_items       0
payment_method    0
created_at        0
dtype: int64

#### There are no missing values.

## Question 1

 On Shopify, we have exactly 100 sneaker shops, and each of these shops sells only one model of shoe. We want to do some analysis of the average order value (AOV). When we look at orders data over a 30 day window, we naively calculate an AOV of $3145.13. Given that we know these shops are selling sneakers, a relatively affordable item, something seems wrong with our analysis. 

- ### a.Think about what could be going wrong with our calculation. Think about a better way to evaluate this data. 

#### Interpretation of the Original AOV

In [67]:
original_AOV=sum(order.order_amount)/len(order.order_amount)
print(original_AOV)

3145.128


#### The original average order value provided above doesn't take the quantities of sneakers in one order into account. The order value can be very high if the quantity is large, thus it is not a good metric to evaluate the values of sneakers.

- ### b.What metric would you report for this dataset?  
- ### c. What is its value?

#### Method 1. Using Another Metric: AIV

In [85]:
ave_item_value=[]
for i in range(0,len(order)):
    ave_item_value.append(order.order_amount[i]/order.total_items[i])

order['ave_item_value']=ave_item_value         #add a new column                                
AIV=sum(ave_item_value)/len(ave_item_value)    #define AIV as the average item value 
print('The average item value AIV is', AIV)

The average item value AIV is 387.7428


#### Conclusion: The first alternative metric is AIV, which calculates the average price of sneakers in different shops, given each of these shops sells only one model of shoe. This metric makes more sense to evaluate the values of sneakers. And the value of the metric is 387.7428.

#### Method 2. Using AOV after Removing Outliers as Metric

#### Detecting Outliers Using IQR Methods

In [83]:
print('The maximum average item value is',max(ave_item_value))

The maximum average item value is 25725.0


In [84]:
print('The minimum average item value is', min(ave_item_value))

The minimum average item value is 90.0


In [93]:
Q1=order['ave_item_value'].quantile(0.25)
Q3=order['ave_item_value'].quantile(0.75)
IQR=Q3-Q1                                       # calculate IQR which describes the middle 50% of values when ordered from lowest to highest
Lower_Whisker = Q1 - 1.5*IQR
Upper_Whisker = Q3 + 1.5*IQR
print(Lower_Whisker, Upper_Whisker)

79.0 223.0


##### The average item values which are outside the range [79,223] are considered outliers, which signify unusual prices for sneakers.

In [86]:
order_new=order[order['ave_item_value']<Upper_Whisker]    #since the minimum AIV is bigger than 79, we only need to consider the upper bound
order_new

,order_id,shop_id,user_id,order_amount,total_items,payment_method,created_at,ave_item_value
0,1,53,746,224,2,cash,2017-03-13 12:36:56,112.0
1,2,92,925,90,1,cash,2017-03-03 17:38:52,90.0
2,3,44,861,144,1,cash,2017-03-14 4:23:56,144.0
3,4,18,935,156,1,credit_card,2017-03-26 12:43:37,156.0
4,5,18,883,156,1,credit_card,2017-03-01 4:35:11,156.0
...,...,...,...,...,...,...,...,...
4995,4996,73,993,330,2,debit,2017-03-30 13:47:17,165.0
4996,4997,48,789,234,2,cash,2017-03-16 20:36:16,117.0
4997,4998,56,867,351,3,cash,2017-03-19 5:42:42,117.0
4998,4999,60,825,354,2,credit_card,2017-03-16 14:51:18,177.0


#### Printing the AOV after Removing Outliers 

In [91]:
AOV=sum(order_new.order_amount)/len(order_new.order_amount)

In [92]:
print('The average order value after removing outliers (very expensive sneakers) is :',AOV)

The average order value after removing outliers (very expensive sneakers) is : 300.1558229655313


#### Conclusion: Another alternative metric is AOV after removing records which have abnormal prices of sneakers. And the value of the metric is 300.1558229655313.